# Create an InterActive Graph of NTRL Lab Analysis Results

In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
import re
from datetime import timedelta
import openpyxl
from sklearn.metrics import mean_squared_error
import plotly.express as px

In [2]:
#used in the analysis results data. cleans up the sampling time entry
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact)
    t = parse(x)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time
def ave_filler(dfname, colname):
    dfcn = dfname[colname]
    try:        
        dfcn = pd.to_numeric(dfcn, errors='coerce')
    except:
        print('error found in' + str(colname))
        pass
    try:
        dfname[colname] = dfcn.where(dfcn.notnull(), other=(dfcn.fillna(method='ffill') + dfcn.fillna(method='bfill'))/2)
    except:
        print('nan not fixed' + str(colname))   
    return dfname
def generate_df_from_analysis(workbookTab, col_sampleDate, col_sampleTime, col_analysis, col_title):
    #get the start and end sample dates
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    Smpl_day_index = list()
    Chem_Element_datelist = list()
    Chem_Element_list = list()
    plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
    #gets the date day from the B column of the analysis file if there is value in the cell
    for i in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
            Smpl_day_index.append(i)
    #get the ending row that has date day
    for j in range(1, workbookTab.max_row):
        if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
            smpl_end_index = j
    for y in range(0, len(Smpl_day_index)):
        if Smpl_day_index[y] != Smpl_day_index[-1]:      
            for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                if workbookTab[str(col_analysis)+str(x)].value is None: 
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))  
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

        if Smpl_day_index[y] == Smpl_day_index[-1]:
            for x in range(Smpl_day_index[y], smpl_end_index+1):
                if workbookTab[str(col_analysis)+str(x)].value is None:   
                    pass
                else:
                    if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                        date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)
                    else:
                        date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                        +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                        Chem_Element_datelist.append(parse(date))
                        chem_element = workbookTab[str(col_analysis)+str(x)].value
                        Chem_Element_list.append(chem_element)

    Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
    Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
    Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
    Chem_Element_df.set_index('DATETIME', inplace=True)

    return Chem_Element_df

def generate_add_trace(figname, df_elemental, axis_bool):
    colname = df_elemental.columns[0]
    figname.add_trace(go.Scatter(x=df_elemental.index, y=df_elemental[colname], name=colname), secondary_y=axis_bool,)
#---------------------------------------

In [3]:
class LabResults():
    def __init__(self, monthyear, filePath):
        #____monthyear format examle is datetime.date(2020,7,1)
        #____filepath example is '\\thgobna001\userdata\THPAL\...\ANALYSIS RESULTS 2020\7) July _2020.xlsx'
        #____add r' at the start of the filepath to apply the regex that allows the use of the filepath as raw
        #____sef.workbook is the uploaded excel workbook of the entire analysis files so it takes time to load
        #____Create an instance of the workbook by giving it a varible name with the sample format: Jan_2020
        
        import openpyxl
        import io
        with open(filePath, "rb") as f:
            in_mem_file = io.BytesIO(f.read())
        
        self.workbook = openpyxl.load_workbook(in_mem_file, data_only=True)
        self.monthyear = monthyear   
        
    def generate_df_from_analysis(self, tabname, list_lab_index):
        #____tabname is a string type name of the tab in the laboratory analysis excel file.
        #____-------Example tabnames: '104PU01', '105TK03', '106TK01'
        #____-------Inside these tabs are the specific analysis results done on the sample taken from the 'tabname' area .
        #____list_lab_index is a list with the sample format is: ['B', 'C', 'E', '104_Pb']
        #____-------the first letter 'B' is the column letter where the day of the month is written as integers 1,2,3..30
        #____-------the second letter 'C' is the column letter where the sampling time is written; e.g. '7:00:00 AM'
        #____-------the third letter 'E' is the column where the results of the specific analysis are listed
        #____-------the last item in the list is the string you'll use as the new title; e.g. '104_Pb'
        
        workbookTab = self.workbook[tabname]
        self.list_lab_index = list_lab_index
        col_sampleDate = self.list_lab_index[0]
        col_sampleTime = self.list_lab_index[1]
        col_analysis = self.list_lab_index[2]
        col_title = self.list_lab_index[3]
        currentmonthyr = self.monthyear
        
        #get the start and end sample dates
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        Smpl_day_index = list()
        Chem_Element_datelist = list()
        Chem_Element_list = list()
        plus_oneday_list = ['0:0:0', '1:0:0', '2:0:0', '3:0:0', '4:0:0', '5:0:0']
        #gets the date day from the B column of the analysis file if there is value in the cell
        for i in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleDate) + str(i)].value) == int or type(workbookTab[str(col_sampleDate) + str(i)].value) == float:
                Smpl_day_index.append(i)
        #get the ending row that has date day
        for j in range(1, workbookTab.max_row):
            if type(workbookTab[str(col_sampleTime) + str(j)].value) == datetime.datetime:
                smpl_end_index = j
        for y in range(0, len(Smpl_day_index)):
            if Smpl_day_index[y] != Smpl_day_index[-1]:      
                for x in range(Smpl_day_index[y], Smpl_day_index[y+1]):
                    if workbookTab[str(col_analysis)+str(x)].value is None: 
                        pass
                    else:
                        try:
                            if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                                date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))  
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                            else:
                                date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))  
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                        except:
                            pass

            if Smpl_day_index[y] == Smpl_day_index[-1]:
                for x in range(Smpl_day_index[y], smpl_end_index+1):
                    if workbookTab[str(col_analysis)+str(x)].value is None:   
                        pass
                    else:
                        try:
                            if time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value)) in plus_oneday_list:
                                date = str((currentmonthyr)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                            else:
                                date = str((currentmonthyr)-datetime.timedelta(1)+datetime.timedelta(workbookTab[str(col_sampleDate)+str(Smpl_day_index[y])].value))\
                                +' '+time_cleaner(str(workbookTab[str(col_sampleTime)+str(x)].value))
                                Chem_Element_datelist.append(parse(date))
                                chem_element = workbookTab[str(col_analysis)+str(x)].value
                                Chem_Element_list.append(chem_element)
                        except:
                            pass

        Chem_Element_list_pair = list(zip(Chem_Element_list, Chem_Element_datelist[0:]))
        Chem_Element_df = pd.DataFrame(Chem_Element_list_pair)
        Chem_Element_df.columns = [col_title, 'DATETIME'] ##check if list title is updated
        Chem_Element_df.set_index('DATETIME', inplace=True)

        return Chem_Element_df

In [4]:
def join_DF(dfsList1):
    df = dfsList1[0]
    for i in range(1,len(dfsList1)):
        df = df.join(dfsList1[i], how='outer')
    return df

def append_monthsDF(dfsListmonthly): #takes the list of dataframes from monthly lab results of one sampling area
    df = dfsListmonthly[0]
    for i in range(1,len(dfsListmonthly)):
        df = df.append(dfsListmonthly[i], ignore_index=True)
    return df

In [5]:
#define the filepaths and the monthyr data for year 2019

currentmonthyr_01_2017 = datetime.date(2017,1,1)
currentmonthyr_02_2017 = datetime.date(2017,2,1)
currentmonthyr_03_2017 = datetime.date(2017,3,1)
currentmonthyr_04_2017 = datetime.date(2017,4,1)
currentmonthyr_05_2017 = datetime.date(2017,5,1)
currentmonthyr_06_2017 = datetime.date(2017,6,1)
currentmonthyr_07_2017 = datetime.date(2017,7,1)
currentmonthyr_08_2017 = datetime.date(2017,8,1)
currentmonthyr_09_2017 = datetime.date(2017,9,1)
currentmonthyr_10_2017 = datetime.date(2017,10,1)
currentmonthyr_11_2017 = datetime.date(2017,11,1)
currentmonthyr_12_2017 = datetime.date(2017,12,1)

currentmonthyr_01_2018 = datetime.date(2018,1,1)
currentmonthyr_02_2018 = datetime.date(2018,2,1)
currentmonthyr_03_2018 = datetime.date(2018,3,1)
currentmonthyr_04_2018 = datetime.date(2018,4,1)
currentmonthyr_05_2018 = datetime.date(2018,5,1)
currentmonthyr_06_2018 = datetime.date(2018,6,1)
currentmonthyr_07_2018 = datetime.date(2018,7,1)
currentmonthyr_08_2018 = datetime.date(2018,8,1)
currentmonthyr_09_2018 = datetime.date(2018,9,1)
currentmonthyr_10_2018 = datetime.date(2018,10,1)
currentmonthyr_11_2018 = datetime.date(2018,11,1)
currentmonthyr_12_2018 = datetime.date(2018,12,1)

currentmonthyr_01_2019 = datetime.date(2019,1,1)
currentmonthyr_02_2019 = datetime.date(2019,2,1)
currentmonthyr_03_2019 = datetime.date(2019,3,1)
currentmonthyr_04_2019 = datetime.date(2019,4,1)
currentmonthyr_05_2019 = datetime.date(2019,5,1)
currentmonthyr_06_2019 = datetime.date(2019,6,1)
currentmonthyr_07_2019 = datetime.date(2019,7,1)
currentmonthyr_08_2019 = datetime.date(2019,8,1)
currentmonthyr_09_2019 = datetime.date(2019,9,1)
currentmonthyr_10_2019 = datetime.date(2019,10,1)
currentmonthyr_11_2019 = datetime.date(2019,11,1)
currentmonthyr_12_2019 = datetime.date(2019,12,1)

currentmonthyr_01_2020 = datetime.date(2020,1,1)
currentmonthyr_02_2020 = datetime.date(2020,2,1)
currentmonthyr_03_2020 = datetime.date(2020,3,1)
currentmonthyr_04_2020 = datetime.date(2020,4,1)
currentmonthyr_05_2020 = datetime.date(2020,5,1)
currentmonthyr_06_2020 = datetime.date(2020,6,1)
currentmonthyr_07_2020 = datetime.date(2020,7,1)
currentmonthyr_08_2020 = datetime.date(2020,8,1)
currentmonthyr_09_2020 = datetime.date(2020,9,1)


#filepath

filepath_01_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\01) January_2017.xlsx'
filepath_02_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\02) February_2017.xlsx'
filepath_03_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\03) March_2017 .xlsx'
filepath_04_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\04) April_2017.xlsx'
filepath_05_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\05) May_2017.xlsx'
filepath_06_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\06) June 2017.xlsx'
filepath_07_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\07) July 2017.xlsx'
filepath_08_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\08) August 2017.xlsx'
filepath_09_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\09) September 2017.xlsx'
filepath_10_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\10) October 2017.xlsx'
filepath_11_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\11) November 2017.xlsx'
filepath_12_2017 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2017\12) December 2017.xlsx'

filepath_01_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\1.) January 2018.xlsx'
filepath_02_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\2.) February 2018.xlsx'
filepath_03_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\3.) March 2018.xlsx'
filepath_04_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\4.) April 2018.xlsx'
filepath_05_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\5.) May 2018.xlsx'
filepath_06_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\6.) June 2018.xlsx'
filepath_07_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\7.) July 2018.xlsx'
filepath_08_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\8.) August 2018.xlsx'
filepath_09_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\9.) September 2018.xlsx'
filepath_10_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\10.) October 2018.xlsx'
filepath_11_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\11.) November 2018.xlsx'
filepath_12_2018 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2018\12.) December 2018.xlsx'

filepath_01_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\1.) January 2019.xlsx'
filepath_02_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\2.) February 2019.xlsx'
filepath_03_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\3.) March 2019.xlsx'
filepath_04_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\4.) April  2019.xlsx'
filepath_05_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\5.) May _2019.xlsx'
filepath_06_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\6.) June  _2019.xlsx'
filepath_07_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\7.) July_2019.xlsx'
filepath_08_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\8.) August_2019.xlsx'
filepath_09_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\9.) September_2019.xlsx'
filepath_10_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\10.) October_ 2019.xlsx'
filepath_11_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\11.) November_ 2019.xlsx'
filepath_12_2019 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2019\12.) December_ 2019.xlsx'

filepath_01_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\01 JANUARY 2020\1) JANUARY_2020.xlsx'
filepath_02_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\02 FEBRUARY 2020\2) FEBRUARY _2020.xlsx'
filepath_03_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\03 MARCH 2020\3) March _2020.xlsx'
filepath_04_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\04 APRIL 2020\4) April _2020.xlsx'
filepath_05_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\05 MAY 2020\5) MAY_2020.xlsx'
filepath_06_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\06 JUNE 2020\6) June _2020.xlsx'
filepath_07_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\07 JULY 2020\7) July _2020.xlsx'
filepath_08_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\08 AUGUST 2020\8) August _2020.xlsx'
filepath_09_2020 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2020\9) September _2020.xlsx'




dateList20172020 = [currentmonthyr_01_2017,currentmonthyr_02_2017,currentmonthyr_03_2017,currentmonthyr_04_2017,currentmonthyr_05_2017,currentmonthyr_06_2017,
                    currentmonthyr_07_2017,currentmonthyr_08_2017,currentmonthyr_09_2017,currentmonthyr_10_2017,currentmonthyr_11_2017,currentmonthyr_12_2017,
                    currentmonthyr_01_2018,currentmonthyr_02_2018,currentmonthyr_03_2018,currentmonthyr_04_2018,currentmonthyr_05_2018,currentmonthyr_06_2018,
                    currentmonthyr_07_2018,currentmonthyr_08_2018,currentmonthyr_09_2018,currentmonthyr_10_2018,currentmonthyr_11_2018,currentmonthyr_12_2018,
                    currentmonthyr_01_2019,currentmonthyr_02_2019,currentmonthyr_03_2019,currentmonthyr_04_2019,currentmonthyr_05_2019,currentmonthyr_06_2019,
                    currentmonthyr_07_2019,currentmonthyr_08_2019,currentmonthyr_09_2019,currentmonthyr_10_2019,currentmonthyr_11_2019,currentmonthyr_12_2019,
                    currentmonthyr_01_2020,currentmonthyr_02_2020,currentmonthyr_03_2020,currentmonthyr_04_2020,currentmonthyr_05_2020,currentmonthyr_06_2020,
                    currentmonthyr_07_2020,currentmonthyr_08_2020,currentmonthyr_09_2020]


filepathList20172020 = [filepath_01_2017,filepath_02_2017,filepath_03_2017,filepath_04_2017,filepath_05_2017,filepath_06_2017,
                        filepath_07_2017,filepath_08_2017,filepath_09_2017,filepath_10_2017,filepath_11_2017,filepath_12_2017,
                        filepath_01_2018,filepath_02_2018,filepath_03_2018,filepath_04_2018,filepath_05_2018,filepath_06_2018,
                        filepath_07_2018,filepath_08_2018,filepath_09_2018,filepath_10_2018,filepath_11_2018,filepath_12_2018,
                        filepath_01_2019,filepath_02_2019,filepath_03_2019,filepath_04_2019,filepath_05_2019,filepath_06_2019,
                        filepath_07_2019,filepath_08_2019,filepath_09_2019,filepath_10_2019,filepath_11_2019,filepath_12_2019,
                        filepath_01_2020,filepath_02_2020,filepath_03_2020,filepath_04_2020,filepath_05_2020,filepath_06_2020,
                        filepath_07_2020,filepath_08_2020,filepath_09_2020]

In [ ]:
print(len(dateList20172020[:17]))
print(len(filepathList20172020[:17]))
print((filepathList20172020[16]))

7

In [6]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2017 to 2017_07 only. The cell location for pH60 and filtration time

ntrl_Pb = ['B', 'C', 'E', '104_Pb']
ntrl_Zn = ['B', 'C', 'F', '104_Zn']
ntrl_Cu = ['B', 'C', 'G', '104_Cu']
ntrl_Ni = ['B', 'C', 'H', '104_Ni']
ntrl_Co = ['B', 'C', 'I', '104_Co']
ntrl_Fe = ['B', 'C', 'J', '104_Fe']
ntrl_Mn = ['B', 'C', 'K', '104_Mn']
ntrl_Cr = ['B', 'C', 'L', '104_Cr']
ntrl_Ca = ['B', 'C', 'M', '104_Ca']
ntrl_Si = ['B', 'C', 'N', '104_Si']
ntrl_Al = ['B', 'C', 'O', '104_Al']
ntrl_Mg = ['B', 'C', 'P', '104_Mg']
ntrl_Fe2 = ['B', 'C', 'R', '104_Fe2']
ntrl_pH60 = ['B', 'C', 'AZ', '104_pH60']
ntrl_ORP = ['B', 'C', 'AJ', '104_ORP']
ntrl_NTU = ['B', 'C', 'AN', '104_NTU']
ntrl_Ft = ['B', 'C', 'AU', '104_Ft']

tk13_Zn = ['B', 'C', 'E', 'tk13_Zn']
tk13_Solids = ['B', 'C', 'BE', 'tk13_solids%']

MStk01_Zn = ['B', 'C', 'F', '106_Zn']
MStk01_Ni = ['B', 'C', 'H', '106_Ni']

for i in range(0,len(dateList20172020[:7])):
    
    monthLab = LabResults(dateList20172020[i], filepathList20172020[i])
    df_month = monthLab.generate_df_from_analysis('104PU01', ntrl_Pb).join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Zn), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Cu), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Ni), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Co), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Fe), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Mn), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Cr), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Ca), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Si), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Al), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Mg), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Fe2), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_pH60), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_ORP), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_NTU), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Ft), how='outer').join(
        monthLab.generate_df_from_analysis('105TK13', tk13_Zn), how='outer').join(
        monthLab.generate_df_from_analysis('105TK13', tk13_Solids), how='outer').join(
        monthLab.generate_df_from_analysis('106TK01', MStk01_Zn), how='outer').join(
        monthLab.generate_df_from_analysis('106TK01', MStk01_Ni), how='outer')
        
    df_month.to_excel(r'C:\Users\v.t.flores\Documents\ZnNi_Ratio_Proj_DataGather_Montly_{}.xlsx'.format(str(dateList20172020[i])))

# reset the variables to an empty list
monthLab = []
df_month = []
#-------------------------------------

In [20]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2017_aug to 2018_0 only. The cell location for pH60 and filtration time

ntrl_Pb = ['B', 'C', 'E', '104_Pb']
ntrl_Zn = ['B', 'C', 'F', '104_Zn']
ntrl_Cu = ['B', 'C', 'G', '104_Cu']
ntrl_Ni = ['B', 'C', 'H', '104_Ni']
ntrl_Co = ['B', 'C', 'I', '104_Co']
ntrl_Fe = ['B', 'C', 'J', '104_Fe']
ntrl_Mn = ['B', 'C', 'K', '104_Mn']
ntrl_Cr = ['B', 'C', 'L', '104_Cr']
ntrl_Ca = ['B', 'C', 'M', '104_Ca']
ntrl_Si = ['B', 'C', 'N', '104_Si']
ntrl_Al = ['B', 'C', 'O', '104_Al']
ntrl_Mg = ['B', 'C', 'P', '104_Mg']
ntrl_Fe2 = ['B', 'C', 'R', '104_Fe2']
ntrl_pH60 = ['B', 'C', 'AZ', '104_pH60']
ntrl_ORP = ['B', 'C', 'AJ', '104_ORP']
ntrl_NTU = ['B', 'C', 'AN', '104_NTU']
ntrl_Ft = ['B', 'C', 'AU', '104_Ft']

tk13_Zn = ['B', 'C', 'E', 'tk13_Zn']
tk13_Solids = ['B', 'C', 'BE', 'tk13_solids%']

MStk01_Zn = ['B', 'C', 'F', '106_Zn']
MStk01_Ni = ['B', 'C', 'H', '106_Ni']

for i in range(7,17):
    
    monthLab = LabResults(dateList20172020[i], filepathList20172020[i])
    df_month = monthLab.generate_df_from_analysis('104PU01', ntrl_Pb).join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Zn), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Cu), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Ni), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Co), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Fe), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Mn), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Cr), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Ca), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Si), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Al), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Mg), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Fe2), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_pH60), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_ORP), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_NTU), how='outer').join(
        monthLab.generate_df_from_analysis('104PU01', ntrl_Ft), how='outer').join(
        monthLab.generate_df_from_analysis('105TK13', tk13_Zn), how='outer').join(
        monthLab.generate_df_from_analysis('105TK13', tk13_Solids), how='outer').join(
        monthLab.generate_df_from_analysis('106TK01', MStk01_Zn), how='outer').join(
        monthLab.generate_df_from_analysis('106TK01', MStk01_Ni), how='outer')
        
    df_month.to_excel(r'C:\Users\v.t.flores\Documents\ZnNi_Ratio_Proj_DataGather_Montly_{}.xlsx'.format(str(dateList20172020[i])))
    
# reset the variables to an empty list
monthLab = []
df_month = []
#-------------------------------------    


In [19]:
print(dateList20172020[0:7])

print(dateList20172020[7])

print(dateList20172020[17])

[datetime.date(2017, 1, 1), datetime.date(2017, 2, 1), datetime.date(2017, 3, 1), datetime.date(2017, 4, 1), datetime.date(2017, 5, 1), datetime.date(2017, 6, 1), datetime.date(2017, 7, 1)]
2017-08-01
2018-06-01


In [27]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2018_06 to 2018_11 only. The cell location for pH60 and filtration time

ntrl_Pb_batch2 = ['B', 'C', 'E', '104_Pb']
ntrl_Zn_batch2 = ['B', 'C', 'F', '104_Zn']
ntrl_Cu_batch2 = ['B', 'C', 'G', '104_Cu']
ntrl_Ni_batch2 = ['B', 'C', 'H', '104_Ni']
ntrl_Co_batch2 = ['B', 'C', 'I', '104_Co']
ntrl_Fe_batch2 = ['B', 'C', 'J', '104_Fe']
ntrl_Mn_batch2 = ['B', 'C', 'K', '104_Mn']
ntrl_Cr_batch2 = ['B', 'C', 'L', '104_Cr']
ntrl_Ca_batch2 = ['B', 'C', 'M', '104_Ca']
ntrl_Si_batch2 = ['B', 'C', 'N', '104_Si']
ntrl_Al_batch2 = ['B', 'C', 'O', '104_Al']
ntrl_Mg_batch2 = ['B', 'C', 'P', '104_Mg']
ntrl_Fe2_batch2 = ['B', 'C', 'R', '104_Fe2']

ntrl_pH60_batch2 = ['B', 'C', 'BB', '104_pH60'] # check
ntrl_ORP_batch2 = ['B', 'C', 'AJ', '104_ORP'] 
ntrl_NTU_batch2 = ['B', 'C', 'AN', '104_NTU'] # check
ntrl_Ft_batch2 = ['B', 'C', 'AW', '104_Ft'] #check


tk13_Zn_batch2 = ['B', 'C', 'E', 'tk13_Zn']
tk13_Solids_batch2 = ['B', 'C', 'BE', 'tk13_solids%'] #check

MStk01_Zn = ['B', 'C', 'F', '106_Zn']
MStk01_Ni = ['B', 'C', 'H', '106_Ni']

for i in range(17,23):
    print(dateList20172020[i])
    monthLab_2 = LabResults(dateList20172020[i], filepathList20172020[i])
    df_month_2 = monthLab_2.generate_df_from_analysis('104PU01', ntrl_Pb_batch2).join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Zn_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Cu_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Ni_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Co_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Fe_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Mn_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Cr_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Ca_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Si_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Al_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Mg_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Fe2_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_pH60_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_ORP_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_NTU_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Ft_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('105TK13', tk13_Zn_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('105TK13', tk13_Solids_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('106TK01', MStk01_Zn), how='outer').join(
        monthLab_2.generate_df_from_analysis('106TK01', MStk01_Ni), how='outer')
    
        
    df_month_2.to_excel(r'C:\Users\v.t.flores\Documents\ZnNi_Ratio_Proj_DataGather_Montly_{}.xlsx'.format(str(dateList20172020[i])))
    print('{} done. time: {}'.format(dateList20172020[i], datetime.datetime.now()))
# reset the variables to an empty list
monthLab_2 = []
df_month_2 = []
#-------------------------------------    

2018-06-01
2018-06-01 done. time: 2020-09-27 09:29:15.668226
2018-07-01
2018-07-01 done. time: 2020-09-27 09:29:51.351080
2018-08-01
2018-08-01 done. time: 2020-09-27 09:30:26.168362
2018-09-01
2018-09-01 done. time: 2020-09-27 09:30:58.833451
2018-10-01
2018-10-01 done. time: 2020-09-27 09:31:33.305791
2018-11-01
2018-11-01 done. time: 2020-09-27 09:32:07.171974


In [ ]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2018_12 to 2018_11 only. The cell location for pH60 and filtration time

ntrl_Pb_batch2 = ['B', 'C', 'E', '104_Pb']
ntrl_Zn_batch2 = ['B', 'C', 'F', '104_Zn']
ntrl_Cu_batch2 = ['B', 'C', 'G', '104_Cu']
ntrl_Ni_batch2 = ['B', 'C', 'H', '104_Ni']
ntrl_Co_batch2 = ['B', 'C', 'I', '104_Co']
ntrl_Fe_batch2 = ['B', 'C', 'J', '104_Fe']
ntrl_Mn_batch2 = ['B', 'C', 'K', '104_Mn']
ntrl_Cr_batch2 = ['B', 'C', 'L', '104_Cr']
ntrl_Ca_batch2 = ['B', 'C', 'M', '104_Ca']
ntrl_Si_batch2 = ['B', 'C', 'N', '104_Si']
ntrl_Al_batch2 = ['B', 'C', 'O', '104_Al']
ntrl_Mg_batch2 = ['B', 'C', 'P', '104_Mg']
ntrl_Fe2_batch2 = ['B', 'C', 'R', '104_Fe2']

ntrl_pH60_batch2 = ['B', 'C', 'BB', '104_pH60'] # check
ntrl_ORP_batch2 = ['B', 'C', 'AJ', '104_ORP'] 
ntrl_NTU_batch2 = ['B', 'C', 'AN', '104_NTU'] # check
ntrl_Ft_batch2 = ['B', 'C', 'AW', '104_Ft'] #check


tk13_Zn_batch2 = ['B', 'C', 'E', 'tk13_Zn']
tk13_Solids_batch2 = ['B', 'C', 'AZ', 'tk13_solids%'] #check

MStk01_Zn = ['B', 'C', 'F', '106_Zn']
MStk01_Ni = ['B', 'C', 'H', '106_Ni']

for i in range(17,23):
    print(dateList20172020[i])
    monthLab_2 = LabResults(dateList20172020[i], filepathList20172020[i])
    df_month_2 = monthLab_2.generate_df_from_analysis('104PU01', ntrl_Pb_batch2).join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Zn_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Cu_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Ni_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Co_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Fe_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Mn_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Cr_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Ca_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Si_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Al_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Mg_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Fe2_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_pH60_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_ORP_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_NTU_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('104PU01', ntrl_Ft_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('105TK13', tk13_Zn_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('105TK13', tk13_Solids_batch2), how='outer').join(
        monthLab_2.generate_df_from_analysis('106TK01', MStk01_Zn), how='outer').join(
        monthLab_2.generate_df_from_analysis('106TK01', MStk01_Ni), how='outer')
    
        
    df_month_2.to_excel(r'C:\Users\v.t.flores\Documents\ZnNi_Ratio_Proj_DataGather_Montly_{}.xlsx'.format(str(dateList20172020[i])))
    print('{} done. time: {}'.format(dateList20172020[i], datetime.datetime.now()))
# reset the variables to an empty list
monthLab_2 = []
df_month_2 = []
#-------------------------------------    

In [29]:
print(dateList20172020[23])
print(dateList20172020[-1])

2018-12-01
2020-09-01


In [30]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2018_12 to 2019 only. The cell location for pH60 and filtration time

ntrl_Pb_3 = ['B', 'C', 'E', '104_Pb']
ntrl_Zn_3 = ['B', 'C', 'F', '104_Zn']
ntrl_Cu_3 = ['B', 'C', 'G', '104_Cu']
ntrl_Ni_3 = ['B', 'C', 'H', '104_Ni']
ntrl_Co_3 = ['B', 'C', 'I', '104_Co']
ntrl_Fe_3 = ['B', 'C', 'J', '104_Fe']
ntrl_Mn_3 = ['B', 'C', 'K', '104_Mn']
ntrl_Cr_3 = ['B', 'C', 'L', '104_Cr']
ntrl_Ca_3 = ['B', 'C', 'M', '104_Ca']
ntrl_Si_3 = ['B', 'C', 'N', '104_Si']
ntrl_Al_3 = ['B', 'C', 'O', '104_Al']
ntrl_Mg_3 = ['B', 'C', 'P', '104_Mg']
ntrl_Fe2_3 = ['B', 'C', 'R', '104_Fe2']
ntrl_pH60_3 = ['B', 'C', 'BB', '104_pH60'] # check
ntrl_ORP_3 = ['B', 'C', 'AJ', '104_ORP'] 
ntrl_NTU_3 = ['B', 'C', 'AN', '104_NTU'] # check
ntrl_Ft_3 = ['B', 'C', 'AW', '104_Ft'] #check

tk13_Zn_3 = ['B', 'C', 'E', 'tk13_Zn']
tk13_Solids_3 = ['B', 'C', 'AZ', 'tk13_solids%'] #check

MStk01_Zn = ['B', 'C', 'F', '106_Zn']
MStk01_Ni = ['B', 'C', 'H', '106_Ni']


for i in range(23,len(dateList20172020)-1):
    print('{} start. time: {}'.format(dateList20172020[i], datetime.datetime.now()))
    monthLab_3 = LabResults(dateList20172020[i], filepathList20172020[i])
    df_month_3 = monthLab_3.generate_df_from_analysis('104PU01', ntrl_Pb_3).join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Zn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cu_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ni_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Co_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cr_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ca_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Si_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Al_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mg_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe2_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_pH60_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_ORP_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_NTU_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ft_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK13', tk13_Zn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK13', tk13_Solids_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Ni), how='outer')
        
    df_month_3.to_excel(r'C:\Users\v.t.flores\Documents\ZnNi_Ratio_Proj_DataGather_Montly_{}.xlsx'.format(str(dateList20172020[i])))
    print('{} done. time: {}'.format(dateList20172020[i], datetime.datetime.now()))
    
# reset the variables to an empty list
monthLab_3 = []
df_month_3 = []
#-------------------------------------        

2018-12-01 start. time: 2020-09-27 09:42:14.929239
2018-12-01 done. time: 2020-09-27 09:42:50.182028
2019-01-01 start. time: 2020-09-27 09:42:50.182028
2019-01-01 done. time: 2020-09-27 09:43:23.399185
2019-02-01 start. time: 2020-09-27 09:43:23.399185
2019-02-01 done. time: 2020-09-27 09:43:55.561903
2019-03-01 start. time: 2020-09-27 09:43:55.561903
2019-03-01 done. time: 2020-09-27 09:44:28.514557
2019-04-01 start. time: 2020-09-27 09:44:28.515564
2019-04-01 done. time: 2020-09-27 09:45:02.347638
2019-05-01 start. time: 2020-09-27 09:45:02.348663
2019-05-01 done. time: 2020-09-27 09:45:37.243933
2019-06-01 start. time: 2020-09-27 09:45:37.243933
2019-06-01 done. time: 2020-09-27 09:46:12.194549
2019-07-01 start. time: 2020-09-27 09:46:12.194549
2019-07-01 done. time: 2020-09-27 09:46:48.234550
2019-08-01 start. time: 2020-09-27 09:46:48.234550
2019-08-01 done. time: 2020-09-27 09:47:23.322922
2019-09-01 start. time: 2020-09-27 09:47:23.322922
2019-09-01 done. time: 2020-09-27 09:47:

In [33]:
#filepath 2016 Laboratory Files

filepath_01_2016 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2016\01) January_2016.xlsx'
filepath_02_2016 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2016\03) February_2016.xlsx'
filepath_03_2016 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2016\05) March_2016.xlsx'
filepath_04_2016 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2016\07) April_2016.xlsx'
filepath_05_2016 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2016\09) May_2016.xlsx'
filepath_06_2016 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2016\11) June_2016.xlsx'
filepath_07_2016 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2016\13) July 2016.xlsx'
filepath_08_2016 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2016\15) August 2016.xlsx'
filepath_09_2016 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2016\17) September_2016.xlsx'
filepath_10_2016 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2016\19) October_2016.xlsx'
filepath_11_2016 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2016\21) November_2016.xlsx'
filepath_12_2016 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2016\23) December_2016.xlsx'


currentmonthyr_01_2016 = datetime.date(2016,1,1)
currentmonthyr_02_2016 = datetime.date(2016,2,1)
currentmonthyr_03_2016 = datetime.date(2016,3,1)
currentmonthyr_04_2016 = datetime.date(2016,4,1)
currentmonthyr_05_2016 = datetime.date(2016,5,1)
currentmonthyr_06_2016 = datetime.date(2016,6,1)
currentmonthyr_07_2016 = datetime.date(2016,7,1)
currentmonthyr_08_2016 = datetime.date(2016,8,1)
currentmonthyr_09_2016 = datetime.date(2016,9,1)
currentmonthyr_10_2016 = datetime.date(2016,10,1)
currentmonthyr_11_2016 = datetime.date(2016,11,1)
currentmonthyr_12_2016 = datetime.date(2016,12,1)

dateList2016 = [currentmonthyr_01_2016,currentmonthyr_02_2016,currentmonthyr_03_2016,currentmonthyr_04_2016,currentmonthyr_05_2016,currentmonthyr_06_2016,
                    currentmonthyr_07_2016,currentmonthyr_08_2016,currentmonthyr_09_2016,currentmonthyr_10_2016,currentmonthyr_11_2016,currentmonthyr_12_2016,
                    ]

filepathList2016 = [filepath_01_2016,filepath_02_2016,filepath_03_2016,filepath_04_2016,filepath_05_2016,filepath_06_2016,
                    filepath_07_2016,filepath_08_2016,filepath_09_2016,filepath_10_2016,filepath_11_2016,filepath_12_2016,
                    ]

### 2016 jan to may

In [34]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2016_01_05 only. The cell location for pH60 and filtration time

ntrl_Pb_3 = ['B', 'C', 'E', '104_Pb']
ntrl_Zn_3 = ['B', 'C', 'F', '104_Zn']
ntrl_Cu_3 = ['B', 'C', 'G', '104_Cu']
ntrl_Ni_3 = ['B', 'C', 'H', '104_Ni']
ntrl_Co_3 = ['B', 'C', 'I', '104_Co']
ntrl_Fe_3 = ['B', 'C', 'J', '104_Fe']
ntrl_Mn_3 = ['B', 'C', 'K', '104_Mn']
ntrl_Cr_3 = ['B', 'C', 'L', '104_Cr']
ntrl_Ca_3 = ['B', 'C', 'M', '104_Ca']
ntrl_Si_3 = ['B', 'C', 'N', '104_Si']
ntrl_Al_3 = ['B', 'C', 'O', '104_Al']
ntrl_Mg_3 = ['B', 'C', 'P', '104_Mg']
ntrl_Fe2_3 = ['B', 'C', 'R', '104_Fe2']
ntrl_pH60_3 = ['B', 'C', 'AY', '104_pH60'] # check
ntrl_ORP_3 = ['B', 'C', 'AI', '104_ORP'] 
ntrl_NTU_3 = ['B', 'C', 'AM', '104_NTU'] # check
ntrl_Ft_3 = ['B', 'C', 'AT', '104_Ft'] #check

tk13_Zn_3 = ['B', 'C', 'E', 'tk13_Zn']
tk13_Solids_3 = ['B', 'C', 'BE', 'tk13_solids%'] #check


DZtk05_Zn = ['B', 'C', 'F', '105_Zn']
MStk01_Zn = ['B', 'C', 'F', '106_Zn']
MStk01_Ni = ['B', 'C', 'H', '106_Ni']


for i in range(0,5):
    
    print('{} start. time: {}'.format(dateList2016[i], datetime.datetime.now()))
    monthLab_ = LabResults(dateList2016[i], filepathList2016[i])
    df_month_ = monthLab_.generate_df_from_analysis('104PU01', ntrl_Pb_3).join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Zn_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Cu_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Ni_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Co_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Fe_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Mn_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Cr_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Ca_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Si_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Al_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Mg_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Fe2_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_pH60_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_ORP_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_NTU_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Ft_3), how='outer').join(
        monthLab_.generate_df_from_analysis('105TK13', tk13_Zn_3), how='outer').join(
        monthLab_.generate_df_from_analysis('105TK13', tk13_Solids_3), how='outer').join(
        monthLab_.generate_df_from_analysis('106TK01', MStk01_Zn), how='outer').join(
        monthLab_.generate_df_from_analysis('105TK05', DZtk05_Zn), how='outer').join(
        monthLab_.generate_df_from_analysis('106TK01', MStk01_Ni), how='outer')
    
        
    df_month_.to_excel(r'C:\Users\v.t.flores\Documents\ZnNi_Ratio_Proj_DataGather_Montly_{}.xlsx'.format(str(dateList2016[i])))
    print('{} done. time: {}'.format(dateList2016[i], datetime.datetime.now()))
    
# reset the variables to an empty list
monthLab_ = []
df_month_ = []
#-------------------------------------     

2016-01-01 start. time: 2020-09-27 09:58:15.561112
2016-01-01 done. time: 2020-09-27 09:58:53.569858
2016-02-01 start. time: 2020-09-27 09:58:53.569858
2016-02-01 done. time: 2020-09-27 09:59:29.720582
2016-03-01 start. time: 2020-09-27 09:59:29.720582
2016-03-01 done. time: 2020-09-27 10:00:04.304135
2016-04-01 start. time: 2020-09-27 10:00:04.304135
2016-04-01 done. time: 2020-09-27 10:00:40.313832
2016-05-01 start. time: 2020-09-27 10:00:40.313832
2016-05-01 done. time: 2020-09-27 10:01:14.200479


### 2016 june to dec

In [39]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2016_06_12 only. The cell location for pH60 and filtration time

ntrl_Pb_3 = ['B', 'C', 'E', '104_Pb']
ntrl_Zn_3 = ['B', 'C', 'F', '104_Zn']
ntrl_Cu_3 = ['B', 'C', 'G', '104_Cu']
ntrl_Ni_3 = ['B', 'C', 'H', '104_Ni']
ntrl_Co_3 = ['B', 'C', 'I', '104_Co']
ntrl_Fe_3 = ['B', 'C', 'J', '104_Fe']
ntrl_Mn_3 = ['B', 'C', 'K', '104_Mn']
ntrl_Cr_3 = ['B', 'C', 'L', '104_Cr']
ntrl_Ca_3 = ['B', 'C', 'M', '104_Ca']
ntrl_Si_3 = ['B', 'C', 'N', '104_Si']
ntrl_Al_3 = ['B', 'C', 'O', '104_Al']
ntrl_Mg_3 = ['B', 'C', 'P', '104_Mg']
ntrl_Fe2_3 = ['B', 'C', 'R', '104_Fe2']
ntrl_pH60_3 = ['B', 'C', 'AZ', '104_pH60'] # check
ntrl_ORP_3 = ['B', 'C', 'AJ', '104_ORP'] 
ntrl_NTU_3 = ['B', 'C', 'AN', '104_NTU'] # check
ntrl_Ft_3 = ['B', 'C', 'AU', '104_Ft'] #check

tk13_Zn_3 = ['B', 'C', 'E', 'tk13_Zn']
tk13_Solids_3 = ['B', 'C', 'BE', 'tk13_solids%'] #check

DZtk05_Zn = ['B', 'C', 'F', '105_Zn']

MStk01_Zn = ['B', 'C', 'F', '106_Zn']
MStk01_Ni = ['B', 'C', 'H', '106_Ni']


for i in range(5,12):
    
    print('{} start. time: {}'.format(dateList2016[i], datetime.datetime.now()))
    monthLab_ = LabResults(dateList2016[i], filepathList2016[i])
    df_month_ = monthLab_.generate_df_from_analysis('104PU01', ntrl_Pb_3).join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Zn_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Cu_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Ni_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Co_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Fe_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Mn_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Cr_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Ca_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Si_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Al_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Mg_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Fe2_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_pH60_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_ORP_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_NTU_3), how='outer').join(
        monthLab_.generate_df_from_analysis('104PU01', ntrl_Ft_3), how='outer').join(
        monthLab_.generate_df_from_analysis('105TK13', tk13_Zn_3), how='outer').join(
        monthLab_.generate_df_from_analysis('105TK13', tk13_Solids_3), how='outer').join(
        monthLab_.generate_df_from_analysis('106TK01', MStk01_Zn), how='outer').join(
        monthLab_.generate_df_from_analysis('105TK05', DZtk05_Zn), how='outer').join(
        monthLab_.generate_df_from_analysis('106TK01', MStk01_Ni), how='outer')
    
        
    df_month_.to_excel(r'C:\Users\v.t.flores\Documents\ZnNi_Ratio_Proj_DataGather_Montly_{}.xlsx'.format(str(dateList2016[i])))
    print('{} done. time: {}'.format(dateList2016[i], datetime.datetime.now()))        
        
# reset the variables to an empty list
monthLab_ = []
df_month_ = []
#-------------------------------------          

2016-06-01 start. time: 2020-09-27 10:07:35.635125
2016-06-01 done. time: 2020-09-27 10:08:21.028540
2016-07-01 start. time: 2020-09-27 10:08:21.028540
2016-07-01 done. time: 2020-09-27 10:09:05.239065
2016-08-01 start. time: 2020-09-27 10:09:05.239065
2016-08-01 done. time: 2020-09-27 10:09:41.816067
2016-09-01 start. time: 2020-09-27 10:09:41.816067
2016-09-01 done. time: 2020-09-27 10:10:17.011108
2016-10-01 start. time: 2020-09-27 10:10:17.011108
2016-10-01 done. time: 2020-09-27 10:10:53.802639
2016-11-01 start. time: 2020-09-27 10:10:53.802639
2016-11-01 done. time: 2020-09-27 10:11:30.727311
2016-12-01 start. time: 2020-09-27 10:11:30.727311
2016-12-01 done. time: 2020-09-27 10:12:07.559010


### 2015

In [41]:
#filepath 2016 Laboratory Files

filepath_01_2015 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2015\1) January_2015.xlsx'
filepath_02_2015 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2015\3) February_2015.xlsx'
filepath_03_2015 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2015\5) March_2015.xlsx'
filepath_04_2015 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2015\7)April _2015.xlsx'
filepath_05_2015 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2015\9) May _2015.xlsx'
filepath_06_2015 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2015\11)June _2015.xlsx'
filepath_07_2015 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2015\13)July _2015.xlsx'
filepath_08_2015 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2015\15) August _2015.xlsx'
filepath_09_2015 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2015\17) September _2015.xlsx'
filepath_10_2015 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2015\19) October _2015.xlsx'
filepath_11_2015 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2015\21) November_2015.xlsx'
filepath_12_2015 = r'\\thgobna001\userdata\THPAL\Department\TAGANITO\SHARED\AnalysisResult\ANALYSIS RESULTS 2015\23) December _2015.xlsx'


currentmonthyr_01_2015 = datetime.date(2015,1,1)
currentmonthyr_02_2015 = datetime.date(2015,2,1)
currentmonthyr_03_2015 = datetime.date(2015,3,1)
currentmonthyr_04_2015 = datetime.date(2015,4,1)
currentmonthyr_05_2015 = datetime.date(2015,5,1)
currentmonthyr_06_2015 = datetime.date(2015,6,1)
currentmonthyr_07_2015 = datetime.date(2015,7,1)
currentmonthyr_08_2015 = datetime.date(2015,8,1)
currentmonthyr_09_2015 = datetime.date(2015,9,1)
currentmonthyr_10_2015 = datetime.date(2015,10,1)
currentmonthyr_11_2015 = datetime.date(2015,11,1)
currentmonthyr_12_2015 = datetime.date(2015,12,1)

dateList2015 = [currentmonthyr_01_2015,currentmonthyr_02_2015,currentmonthyr_03_2015,currentmonthyr_04_2015,currentmonthyr_05_2015,currentmonthyr_06_2015,
                    currentmonthyr_07_2015,currentmonthyr_08_2015,currentmonthyr_09_2015,currentmonthyr_10_2015,currentmonthyr_11_2015,currentmonthyr_12_2015,
                    ]

filepathList2015 = [filepath_01_2015,filepath_02_2015,filepath_03_2015,filepath_04_2015,filepath_05_2015,filepath_06_2015,
                    filepath_07_2015,filepath_08_2015,filepath_09_2015,filepath_10_2015,filepath_11_2015,filepath_12_2015,
                    ]

### 2015 Jan to May

In [42]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2015_01_05 only. The cell location for pH60 and filtration time

ntrl_Pb_3 = ['B', 'C', 'E', '104_Pb']
ntrl_Zn_3 = ['B', 'C', 'F', '104_Zn']
ntrl_Cu_3 = ['B', 'C', 'G', '104_Cu']
ntrl_Ni_3 = ['B', 'C', 'H', '104_Ni']
ntrl_Co_3 = ['B', 'C', 'I', '104_Co']
ntrl_Fe_3 = ['B', 'C', 'J', '104_Fe']
ntrl_Mn_3 = ['B', 'C', 'K', '104_Mn']
ntrl_Cr_3 = ['B', 'C', 'L', '104_Cr']
ntrl_Ca_3 = ['B', 'C', 'M', '104_Ca']
ntrl_Si_3 = ['B', 'C', 'N', '104_Si']
ntrl_Al_3 = ['B', 'C', 'O', '104_Al']
ntrl_Mg_3 = ['B', 'C', 'P', '104_Mg']

ntrl_Fe2_3 = ['B', 'C', 'R', '104_Fe2']
ntrl_pH60_3 = ['B', 'C', 'AZ', '104_pH60'] # check
ntrl_ORP_3 = ['B', 'C', 'AI', '104_ORP'] 
ntrl_NTU_3 = ['B', 'C', 'AM', '104_NTU'] # check
ntrl_Ft_3 = ['B', 'C', 'AT', '104_Ft'] #check

#tk13_Zn_3 = ['B', 'C', 'E', 'tk13_Zn']
tk13_Solids_3 = ['B', 'C', 'AJ', 'tk13_solids%'] #check

DZtk05_Zn = ['B', 'C', 'F', '105_Zn']

MStk01_Zn = ['B', 'C', 'F', '106_Zn']
MStk01_Ni = ['B', 'C', 'H', '106_Ni']


for i in range(0,5):
    
    print('{} start. time: {}'.format(dateList2015[i], datetime.datetime.now()))
    monthLab_3 = LabResults(dateList2015[i], filepathList2015[i])
    df_month_3 = monthLab_3.generate_df_from_analysis('104PU01', ntrl_Pb_3).join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Zn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cu_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ni_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Co_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cr_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ca_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Si_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Al_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mg_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe2_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_pH60_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_ORP_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_NTU_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ft_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK13', tk13_Solids_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', DZtk05_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Ni), how='outer')
        
    df_month_3.to_excel(r'C:\Users\v.t.flores\Documents\ZnNi_Ratio_Proj_DataGather_Montly_{}.xlsx'.format(str(dateList2015[i])))
    print('{} done. time: {}'.format(dateList2015[i], datetime.datetime.now()))
# reset the variables to an empty list
monthLab_3 = []
df_month_3 = []
#-------------------------------------       

2015-01-01 start. time: 2020-09-27 10:12:31.845879
2015-01-01 done. time: 2020-09-27 10:13:08.182461
2015-02-01 start. time: 2020-09-27 10:13:08.182461
2015-02-01 done. time: 2020-09-27 10:13:43.164425
2015-03-01 start. time: 2020-09-27 10:13:43.164425
2015-03-01 done. time: 2020-09-27 10:14:18.478360
2015-04-01 start. time: 2020-09-27 10:14:18.479389
2015-04-01 done. time: 2020-09-27 10:14:53.497887
2015-05-01 start. time: 2020-09-27 10:14:53.497887
2015-05-01 done. time: 2020-09-27 10:15:30.596618


In [43]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2015_06_07 only. The cell location for pH60 and filtration time

ntrl_Pb_3 = ['B', 'C', 'E', '104_Pb']
ntrl_Zn_3 = ['B', 'C', 'F', '104_Zn']
ntrl_Cu_3 = ['B', 'C', 'G', '104_Cu']
ntrl_Ni_3 = ['B', 'C', 'H', '104_Ni']
ntrl_Co_3 = ['B', 'C', 'I', '104_Co']
ntrl_Fe_3 = ['B', 'C', 'J', '104_Fe']
ntrl_Mn_3 = ['B', 'C', 'K', '104_Mn']
ntrl_Cr_3 = ['B', 'C', 'L', '104_Cr']
ntrl_Ca_3 = ['B', 'C', 'M', '104_Ca']
ntrl_Si_3 = ['B', 'C', 'N', '104_Si']
ntrl_Al_3 = ['B', 'C', 'O', '104_Al']
ntrl_Mg_3 = ['B', 'C', 'P', '104_Mg']

ntrl_Fe2_3 = ['B', 'C', 'R', '104_Fe2']
ntrl_pH60_3 = ['B', 'C', 'AZ', '104_pH60'] # check
ntrl_ORP_3 = ['B', 'C', 'AI', '104_ORP'] 
ntrl_NTU_3 = ['B', 'C', 'AM', '104_NTU'] # check
ntrl_Ft_3 = ['B', 'C', 'AT', '104_Ft'] #check

tk13_Zn_3 = ['B', 'C', 'D', 'tk13_Zn']
tk13_Solids_3 = ['B', 'C', 'BA', 'tk13_solids%'] #check

DZtk05_Zn = ['B', 'C', 'F', '105_Zn']
MStk01_Zn = ['B', 'C', 'F', '106_Zn']
MStk01_Ni = ['B', 'C', 'H', '106_Ni']


for i in range(5,7):
    
    print('{} start. time: {}'.format(dateList2015[i], datetime.datetime.now()))
    monthLab_3 = LabResults(dateList2015[i], filepathList2015[i])
    df_month_3 = monthLab_3.generate_df_from_analysis('104PU01', ntrl_Pb_3).join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Zn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cu_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ni_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Co_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cr_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ca_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Si_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Al_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mg_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe2_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_pH60_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_ORP_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_NTU_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ft_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK13', tk13_Zn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK13', tk13_Solids_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', DZtk05_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Ni), how='outer')
        
    df_month_3.to_excel(r'C:\Users\v.t.flores\Documents\ZnNi_Ratio_Proj_DataGather_Montly_{}.xlsx'.format(str(dateList2015[i])))
    print('{} done. time: {}'.format(dateList2015[i], datetime.datetime.now()))
# reset the variables to an empty list
monthLab_3 = []
df_month_3 = []
#-------------------------------------       

2015-06-01 start. time: 2020-09-27 10:17:07.547601
2015-06-01 done. time: 2020-09-27 10:17:44.150780
2015-07-01 start. time: 2020-09-27 10:17:44.150780
2015-07-01 done. time: 2020-09-27 10:18:20.084824


In [44]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2016_08_10 only. The cell location for pH60 and filtration time

ntrl_Pb_3 = ['B', 'C', 'E', '104_Pb']
ntrl_Zn_3 = ['B', 'C', 'F', '104_Zn']
ntrl_Cu_3 = ['B', 'C', 'G', '104_Cu']
ntrl_Ni_3 = ['B', 'C', 'H', '104_Ni']
ntrl_Co_3 = ['B', 'C', 'I', '104_Co']
ntrl_Fe_3 = ['B', 'C', 'J', '104_Fe']
ntrl_Mn_3 = ['B', 'C', 'K', '104_Mn']
ntrl_Cr_3 = ['B', 'C', 'L', '104_Cr']
ntrl_Ca_3 = ['B', 'C', 'M', '104_Ca']
ntrl_Si_3 = ['B', 'C', 'N', '104_Si']
ntrl_Al_3 = ['B', 'C', 'O', '104_Al']
ntrl_Mg_3 = ['B', 'C', 'P', '104_Mg']

ntrl_Fe2_3 = ['B', 'C', 'R', '104_Fe2']
ntrl_pH60_3 = ['B', 'C', 'AZ', '104_pH60'] # check
ntrl_ORP_3 = ['B', 'C', 'AI', '104_ORP'] 
ntrl_NTU_3 = ['B', 'C', 'AM', '104_NTU'] # check
ntrl_Ft_3 = ['B', 'C', 'AT', '104_Ft'] #check

tk13_Zn_3 = ['B', 'C', 'E', 'tk13_Zn']
tk13_Solids_3 = ['B', 'C', 'BC', 'tk13_solids%'] #check

DZtk05_Zn = ['B', 'C', 'F', '105_Zn']

MStk01_Zn = ['B', 'C', 'F', '106_Zn']
MStk01_Ni = ['B', 'C', 'H', '106_Ni']


for i in range(7,10):
    
    print('{} start. time: {}'.format(dateList2015[i], datetime.datetime.now()))
    monthLab_3 = LabResults(dateList2015[i], filepathList2015[i])
    df_month_3 = monthLab_3.generate_df_from_analysis('104PU01', ntrl_Pb_3).join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Zn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cu_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ni_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Co_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cr_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ca_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Si_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Al_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mg_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe2_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_pH60_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_ORP_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_NTU_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ft_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK13', tk13_Zn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK13', tk13_Solids_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', DZtk05_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Ni), how='outer')
        
    df_month_3.to_excel(r'C:\Users\v.t.flores\Documents\ZnNi_Ratio_Proj_DataGather_Montly_{}.xlsx'.format(str(dateList2015[i])))
    print('{} done. time: {}'.format(dateList2015[i], datetime.datetime.now()))
# reset the variables to an empty list
monthLab_3 = []
df_month_3 = []
#-------------------------------------       

2015-08-01 start. time: 2020-09-27 10:20:12.856021
2015-08-01 done. time: 2020-09-27 10:20:49.884574
2015-09-01 start. time: 2020-09-27 10:20:49.884574
2015-09-01 done. time: 2020-09-27 10:21:24.007650
2015-10-01 start. time: 2020-09-27 10:21:24.007650
2015-10-01 done. time: 2020-09-27 10:22:00.609408


In [45]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2016_11 only. The cell location for pH60 and filtration time

ntrl_Pb_3 = ['B', 'C', 'E', '104_Pb']
ntrl_Zn_3 = ['B', 'C', 'F', '104_Zn']
ntrl_Cu_3 = ['B', 'C', 'G', '104_Cu']
ntrl_Ni_3 = ['B', 'C', 'H', '104_Ni']
ntrl_Co_3 = ['B', 'C', 'I', '104_Co']
ntrl_Fe_3 = ['B', 'C', 'J', '104_Fe']
ntrl_Mn_3 = ['B', 'C', 'K', '104_Mn']
ntrl_Cr_3 = ['B', 'C', 'L', '104_Cr']
ntrl_Ca_3 = ['B', 'C', 'M', '104_Ca']
ntrl_Si_3 = ['B', 'C', 'N', '104_Si']
ntrl_Al_3 = ['B', 'C', 'O', '104_Al']
ntrl_Mg_3 = ['B', 'C', 'P', '104_Mg']

ntrl_Fe2_3 = ['B', 'C', 'R', '104_Fe2']
ntrl_pH60_3 = ['B', 'C', 'AZ', '104_pH60'] # check
ntrl_ORP_3 = ['B', 'C', 'AI', '104_ORP'] 
ntrl_NTU_3 = ['B', 'C', 'AM', '104_NTU'] # check
ntrl_Ft_3 = ['B', 'C', 'AT', '104_Ft'] #check

tk13_Zn_3 = ['B', 'C', 'E', 'tk13_Zn']
tk13_Solids_3 = ['B', 'C', 'BD', 'tk13_solids%'] #check

DZtk05_Zn = ['B', 'C', 'F', '105_Zn']

MStk01_Zn = ['B', 'C', 'F', '106_Zn']
MStk01_Ni = ['B', 'C', 'H', '106_Ni']

for i in range(10,11):
    
    print('{} start. time: {}'.format(dateList2015[i], datetime.datetime.now()))
    monthLab_3 = LabResults(dateList2015[i], filepathList2015[i])
    df_month_3 = monthLab_3.generate_df_from_analysis('104PU01', ntrl_Pb_3).join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Zn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cu_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ni_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Co_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cr_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ca_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Si_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Al_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mg_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe2_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_pH60_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_ORP_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_NTU_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ft_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK13', tk13_Zn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK13', tk13_Solids_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', DZtk05_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Ni), how='outer')
        
    df_month_3.to_excel(r'C:\Users\v.t.flores\Documents\ZnNi_Ratio_Proj_DataGather_Montly_{}.xlsx'.format(str(dateList2015[i])))
    print('{} done. time: {}'.format(dateList2015[i], datetime.datetime.now()))
# reset the variables to an empty list
monthLab_3 = []
df_month_3 = []
#-------------------------------------       

2015-11-01 start. time: 2020-09-27 10:22:00.627538
2015-11-01 done. time: 2020-09-27 10:22:39.617063


In [46]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2016_11 only. The cell location for pH60 and filtration time

ntrl_Pb_3 = ['B', 'C', 'E', '104_Pb']
ntrl_Zn_3 = ['B', 'C', 'F', '104_Zn']
ntrl_Cu_3 = ['B', 'C', 'G', '104_Cu']
ntrl_Ni_3 = ['B', 'C', 'H', '104_Ni']
ntrl_Co_3 = ['B', 'C', 'I', '104_Co']
ntrl_Fe_3 = ['B', 'C', 'J', '104_Fe']
ntrl_Mn_3 = ['B', 'C', 'K', '104_Mn']
ntrl_Cr_3 = ['B', 'C', 'L', '104_Cr']
ntrl_Ca_3 = ['B', 'C', 'M', '104_Ca']
ntrl_Si_3 = ['B', 'C', 'N', '104_Si']
ntrl_Al_3 = ['B', 'C', 'O', '104_Al']
ntrl_Mg_3 = ['B', 'C', 'P', '104_Mg']

ntrl_Fe2_3 = ['B', 'C', 'R', '104_Fe2']
ntrl_pH60_3 = ['B', 'C', 'AY', '104_pH60'] # check
ntrl_ORP_3 = ['B', 'C', 'AI', '104_ORP'] 
ntrl_NTU_3 = ['B', 'C', 'AM', '104_NTU'] # check
ntrl_Ft_3 = ['B', 'C', 'AT', '104_Ft'] #check

tk13_Zn_3 = ['B', 'C', 'E', 'tk13_Zn']
tk13_Solids_3 = ['B', 'C', 'BE', 'tk13_solids%'] #check

MStk01_Zn = ['B', 'C', 'F', '106_Zn']
DZtk05_Zn = ['B', 'C', 'F', '105_Zn']

MStk01_Ni = ['B', 'C', 'H', '106_Ni']

for i in range(11,12):
    
    print('{} start. time: {}'.format(dateList2015[i], datetime.datetime.now()))
    monthLab_3 = LabResults(dateList2015[i], filepathList2015[i])
    df_month_3 = monthLab_3.generate_df_from_analysis('104PU01', ntrl_Pb_3).join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Zn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cu_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ni_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Co_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cr_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ca_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Si_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Al_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mg_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe2_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_pH60_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_ORP_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_NTU_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ft_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK13', tk13_Zn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK13', tk13_Solids_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', DZtk05_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Ni), how='outer')
        
    df_month_3.to_excel(r'C:\Users\v.t.flores\Documents\ZnNi_Ratio_Proj_DataGather_Montly_{}.xlsx'.format(str(dateList2015[i])))
    print('{} done. time: {}'.format(dateList2015[i], datetime.datetime.now()))
# reset the variables to an empty list
monthLab_3 = []
df_month_3 = []
#-------------------------------------       

2015-12-01 start. time: 2020-09-27 10:22:39.635043
2015-12-01 done. time: 2020-09-27 10:23:16.180487


In [47]:
#filepath 2014 Laboratory Files

filepath_01_2014 = r'\\THGOBNA002\thpal\05 Production\800 Personal Folder\analysis file convert 2014\1) January_2014.xlsx'
filepath_02_2014 = r'\\THGOBNA002\thpal\05 Production\800 Personal Folder\analysis file convert 2014\3) February_2014.xlsx'
filepath_03_2014 = r'\\THGOBNA002\thpal\05 Production\800 Personal Folder\analysis file convert 2014\5) March_2014.xlsx'
filepath_04_2014 = r'\\THGOBNA002\thpal\05 Production\800 Personal Folder\analysis file convert 2014\7)April_2014.xlsx'
filepath_05_2014 = r'\\THGOBNA002\thpal\05 Production\800 Personal Folder\analysis file convert 2014\9) May_2014.xlsx'
filepath_06_2014 = r'\\THGOBNA002\thpal\05 Production\800 Personal Folder\analysis file convert 2014\11) June_2014.xlsx'
filepath_07_2014 = r'\\THGOBNA002\thpal\05 Production\800 Personal Folder\analysis file convert 2014\13) July_2014.xlsx'
filepath_08_2014 = r'\\THGOBNA002\thpal\05 Production\800 Personal Folder\analysis file convert 2014\15) August_2014.xlsx'
filepath_09_2014 = r'\\THGOBNA002\thpal\05 Production\800 Personal Folder\analysis file convert 2014\17) September_2014.xlsx'
filepath_10_2014 = r'\\THGOBNA002\thpal\05 Production\800 Personal Folder\analysis file convert 2014\19) October_2014.xlsx'
filepath_11_2014 = r'\\THGOBNA002\thpal\05 Production\800 Personal Folder\analysis file convert 2014\21) November_2014.xlsx'
filepath_12_2014 = r'\\THGOBNA002\thpal\05 Production\800 Personal Folder\analysis file convert 2014\23) December_2014.xlsx'


currentmonthyr_01_2014 = datetime.date(2014,1,1)
currentmonthyr_02_2014 = datetime.date(2014,2,1)
currentmonthyr_03_2014 = datetime.date(2014,3,1)
currentmonthyr_04_2014 = datetime.date(2014,4,1)
currentmonthyr_05_2014 = datetime.date(2014,5,1)
currentmonthyr_06_2014 = datetime.date(2014,6,1)
currentmonthyr_07_2014 = datetime.date(2014,7,1)
currentmonthyr_08_2014 = datetime.date(2014,8,1)
currentmonthyr_09_2014 = datetime.date(2014,9,1)
currentmonthyr_10_2014 = datetime.date(2014,10,1)
currentmonthyr_11_2014 = datetime.date(2014,11,1)
currentmonthyr_12_2014 = datetime.date(2014,12,1)

dateList2014 = [currentmonthyr_01_2014,currentmonthyr_02_2014,currentmonthyr_03_2014,currentmonthyr_04_2014,currentmonthyr_05_2014,currentmonthyr_06_2014,
                    currentmonthyr_07_2014,currentmonthyr_08_2014,currentmonthyr_09_2014,currentmonthyr_10_2014,currentmonthyr_11_2014,currentmonthyr_12_2014
                    ]

filepathList2014 = [filepath_01_2014,filepath_02_2014,filepath_03_2014,filepath_04_2014,filepath_05_2014,filepath_06_2014,
                    filepath_07_2014,filepath_08_2014,filepath_09_2014,filepath_10_2014,filepath_11_2014,filepath_12_2014]

### 2014 jan to oct

In [49]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2014_01_10 only. The cell location for pH60 and filtration time

ntrl_Pb_3 = ['B', 'C', 'E', '104_Pb']
ntrl_Zn_3 = ['B', 'C', 'F', '104_Zn']
ntrl_Cu_3 = ['B', 'C', 'G', '104_Cu']
ntrl_Ni_3 = ['B', 'C', 'H', '104_Ni']
ntrl_Co_3 = ['B', 'C', 'I', '104_Co']
ntrl_Fe_3 = ['B', 'C', 'J', '104_Fe']
ntrl_Mn_3 = ['B', 'C', 'K', '104_Mn']
ntrl_Cr_3 = ['B', 'C', 'L', '104_Cr']
ntrl_Ca_3 = ['B', 'C', 'M', '104_Ca']
ntrl_Si_3 = ['B', 'C', 'N', '104_Si']
ntrl_Al_3 = ['B', 'C', 'O', '104_Al']
ntrl_Mg_3 = ['B', 'C', 'P', '104_Mg']

ntrl_Fe2_3 = ['B', 'C', 'R', '104_Fe2']
ntrl_pH60_3 = ['B', 'C', 'AG', '104_pH60'] # check
ntrl_ORP_3 = ['B', 'C', 'AH', '104_ORP'] 
ntrl_NTU_3 = ['B', 'C', 'AM', '104_NTU'] # check
ntrl_Ft_3 = ['B', 'C', 'AT', '104_Ft'] #check

#tk13_Zn_3 = ['B', 'C', 'E', 'tk13_Zn']
tk13_Solids_3 = ['B', 'C', 'AJ', 'tk13_solids%'] #check

MStk01_Zn = ['B', 'C', 'F', '106_Zn']
DZtk05_Zn = ['B', 'C', 'F', '105_Zn']
MStk01_Ni = ['B', 'C', 'H', '106_Ni']




for i in range(0,10):
    
    print('{} start. time: {}'.format(dateList2014[i], datetime.datetime.now()))
    monthLab_3 = LabResults(dateList2014[i], filepathList2014[i])
    df_month_3 = monthLab_3.generate_df_from_analysis('104PU01', ntrl_Pb_3).join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Zn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cu_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ni_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Co_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cr_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ca_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Si_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Al_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mg_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe2_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_pH60_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_ORP_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_NTU_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ft_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK13', tk13_Solids_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', DZtk05_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Ni), how='outer')
        
    df_month_3.to_excel(r'C:\Users\v.t.flores\Documents\ZnNi_Ratio_Proj_DataGather_Montly_{}.xlsx'.format(str(dateList2014[i])))
    print('{} done. time: {}'.format(dateList2014[i], datetime.datetime.now()))
# reset the variables to an empty list
monthLab_3 = []
df_month_3 = []
#-------------------------------------           

2014-01-01 start. time: 2020-09-27 10:28:08.395417
2014-01-01 done. time: 2020-09-27 10:28:42.299607
2014-02-01 start. time: 2020-09-27 10:28:42.299607
2014-02-01 done. time: 2020-09-27 10:29:14.556438
2014-03-01 start. time: 2020-09-27 10:29:14.556438
2014-03-01 done. time: 2020-09-27 10:29:50.815316
2014-04-01 start. time: 2020-09-27 10:29:50.815316
2014-04-01 done. time: 2020-09-27 10:30:23.414082
2014-05-01 start. time: 2020-09-27 10:30:23.415078
2014-05-01 done. time: 2020-09-27 10:30:56.841231
2014-06-01 start. time: 2020-09-27 10:30:56.841231
2014-06-01 done. time: 2020-09-27 10:31:30.163320
2014-07-01 start. time: 2020-09-27 10:31:30.163320
2014-07-01 done. time: 2020-09-27 10:32:05.763822
2014-08-01 start. time: 2020-09-27 10:32:05.764842
2014-08-01 done. time: 2020-09-27 10:32:40.655546
2014-09-01 start. time: 2020-09-27 10:32:40.655546
2014-09-01 done. time: 2020-09-27 10:33:16.370798
2014-10-01 start. time: 2020-09-27 10:33:16.370798
2014-10-01 done. time: 2020-09-27 10:33:

In [50]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2014_11 only. The cell location for pH60 and filtration time

ntrl_Pb_3 = ['B', 'C', 'E', '104_Pb']
ntrl_Zn_3 = ['B', 'C', 'F', '104_Zn']
ntrl_Cu_3 = ['B', 'C', 'G', '104_Cu']
ntrl_Ni_3 = ['B', 'C', 'H', '104_Ni']
ntrl_Co_3 = ['B', 'C', 'I', '104_Co']
ntrl_Fe_3 = ['B', 'C', 'J', '104_Fe']
ntrl_Mn_3 = ['B', 'C', 'K', '104_Mn']
ntrl_Cr_3 = ['B', 'C', 'L', '104_Cr']
ntrl_Ca_3 = ['B', 'C', 'M', '104_Ca']
ntrl_Si_3 = ['B', 'C', 'N', '104_Si']
ntrl_Al_3 = ['B', 'C', 'O', '104_Al']
ntrl_Mg_3 = ['B', 'C', 'P', '104_Mg']

ntrl_Fe2_3 = ['B', 'C', 'R', '104_Fe2']
ntrl_pH60_3 = ['B', 'C', 'AH', '104_pH60'] # check
ntrl_ORP_3 = ['B', 'C', 'AI', '104_ORP'] 
ntrl_NTU_3 = ['B', 'C', 'AM', '104_NTU'] # check
ntrl_Ft_3 = ['B', 'C', 'AT', '104_Ft'] #check

#tk13_Zn_3 = ['B', 'C', 'E', 'tk13_Zn']
tk13_Solids_3 = ['B', 'C', 'AJ', 'tk13_solids%'] #check

MStk01_Zn = ['B', 'C', 'F', '106_Zn']
DZtk05_Zn = ['B', 'C', 'F', '105_Zn']
MStk01_Ni = ['B', 'C', 'H', '106_Ni']


for i in range(10,11):
    
    print('{} start. time: {}'.format(dateList2014[i], datetime.datetime.now()))
    monthLab_3 = LabResults(dateList2014[i], filepathList2014[i])
    df_month_3 = monthLab_3.generate_df_from_analysis('104PU01', ntrl_Pb_3).join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Zn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cu_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ni_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Co_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cr_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ca_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Si_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Al_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mg_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe2_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_pH60_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_ORP_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_NTU_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ft_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK13', tk13_Solids_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', DZtk05_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Ni), how='outer')
        
    df_month_3.to_excel(r'C:\Users\v.t.flores\Documents\ZnNi_Ratio_Proj_DataGather_Montly_{}.xlsx'.format(str(dateList2014[i])))
    print('{} done. time: {}'.format(dateList2014[i], datetime.datetime.now()))
# reset the variables to an empty list
monthLab_3 = []
df_month_3 = []
#-------------------------------------       

2014-11-01 start. time: 2020-09-27 10:33:56.948321
2014-11-01 done. time: 2020-09-27 10:34:34.830981


In [51]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2014_12 only. The cell location for pH60 and filtration time

ntrl_Pb_3 = ['B', 'C', 'E', '104_Pb']
ntrl_Zn_3 = ['B', 'C', 'F', '104_Zn']
ntrl_Cu_3 = ['B', 'C', 'G', '104_Cu']
ntrl_Ni_3 = ['B', 'C', 'H', '104_Ni']
ntrl_Co_3 = ['B', 'C', 'I', '104_Co']
ntrl_Fe_3 = ['B', 'C', 'J', '104_Fe']
ntrl_Mn_3 = ['B', 'C', 'K', '104_Mn']
ntrl_Cr_3 = ['B', 'C', 'L', '104_Cr']
ntrl_Ca_3 = ['B', 'C', 'M', '104_Ca']
ntrl_Si_3 = ['B', 'C', 'N', '104_Si']
ntrl_Al_3 = ['B', 'C', 'O', '104_Al']
ntrl_Mg_3 = ['B', 'C', 'P', '104_Mg']

ntrl_Fe2_3 = ['B', 'C', 'R', '104_Fe2']
ntrl_pH60_3 = ['B', 'C', 'AZ', '104_pH60'] # check
ntrl_ORP_3 = ['B', 'C', 'AI', '104_ORP'] 
ntrl_NTU_3 = ['B', 'C', 'AM', '104_NTU'] # check
ntrl_Ft_3 = ['B', 'C', 'AT', '104_Ft'] #check

#tk13_Zn_3 = ['B', 'C', 'E', 'tk13_Zn']
tk13_Solids_3 = ['B', 'C', 'AJ', 'tk13_solids%'] #check

MStk01_Zn = ['B', 'C', 'F', '106_Zn']
DZtk05_Zn = ['B', 'C', 'F', '105_Zn']
MStk01_Ni = ['B', 'C', 'H', '106_Ni']


for i in range(11,12):
    
    print('{} start. time: {}'.format(dateList2014[i], datetime.datetime.now()))
    monthLab_3 = LabResults(dateList2014[i], filepathList2014[i])
    df_month_3 = monthLab_3.generate_df_from_analysis('104PU01', ntrl_Pb_3).join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Zn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cu_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ni_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Co_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mn_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Cr_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ca_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Si_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Al_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Mg_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Fe2_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_pH60_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_ORP_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_NTU_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('104PU01', ntrl_Ft_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK13', tk13_Solids_3), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('105TK05', DZtk05_Zn), how='outer').join(
        monthLab_3.generate_df_from_analysis('106TK01', MStk01_Ni), how='outer')
        
    df_month_3.to_excel(r'C:\Users\v.t.flores\Documents\ZnNi_Ratio_Proj_DataGather_Montly_{}.xlsx'.format(str(dateList2014[i])))
    print('{} done. time: {}'.format(dateList2014[i], datetime.datetime.now()))
# reset the variables to an empty list
monthLab_3 = []
df_month_3 = []
#-------------------------------------       

2014-12-01 start. time: 2020-09-27 10:34:34.847962
2014-12-01 done. time: 2020-09-27 10:35:12.206556


In [53]:


#dateList2016 = [currentmonthyr_01_2013,currentmonthyr_02_2013,currentmonthyr_03_2013,currentmonthyr_04_2013,currentmonthyr_05_2013,currentmonthyr_06_2013,
#                    currentmonthyr_07_2013,currentmonthyr_08_2013,currentmonthyr_09_2013,currentmonthyr_10_2013,currentmonthyr_11_2013,currentmonthyr_12_2013,
#                    ]

#filepathList2016 = [filepath_01_2013,filepath_02_2013,filepath_03_2013,filepath_04_2013,filepath_05_2013,filepath_06_2013,
#                    filepath_07_2013,filepath_08_2013,filepath_09_2013,filepath_10_2013,filepath_11_2013,filepath_12_2013,
#                    ]

dateList2014_2016 = [currentmonthyr_01_2014,
                     currentmonthyr_02_2014,
                     currentmonthyr_03_2014,
                     currentmonthyr_04_2014,
                     currentmonthyr_05_2014,
                     currentmonthyr_06_2014,
                     currentmonthyr_07_2014,
                     currentmonthyr_08_2014,
                     currentmonthyr_09_2014,
                     currentmonthyr_10_2014,
                     currentmonthyr_11_2014,
                     currentmonthyr_12_2014,
                     currentmonthyr_01_2015,
                     currentmonthyr_02_2015,
                     currentmonthyr_03_2015,
                     currentmonthyr_04_2015,
                     currentmonthyr_05_2015,
                     currentmonthyr_06_2015,
                     currentmonthyr_07_2015,
                     currentmonthyr_08_2015,
                     currentmonthyr_09_2015,
                     currentmonthyr_10_2015,
                     currentmonthyr_11_2015,
                     currentmonthyr_12_2015,
                     currentmonthyr_01_2016,
                     currentmonthyr_02_2016,
                     currentmonthyr_03_2016,
                     currentmonthyr_04_2016,
                     currentmonthyr_05_2016,
                     currentmonthyr_06_2016,
                     currentmonthyr_07_2016,
                     currentmonthyr_08_2016,
                     currentmonthyr_09_2016,
                     currentmonthyr_10_2016,
                     currentmonthyr_11_2016,
                     currentmonthyr_12_2016
                    ]

filepathList2014_2016 = [filepath_01_2014,
                         filepath_02_2014,
                         filepath_03_2014,
                         filepath_04_2014,
                         filepath_05_2014,
                         filepath_06_2014,
                         filepath_07_2014,
                         filepath_08_2014,
                         filepath_09_2014,
                         filepath_10_2014,
                         filepath_11_2014,
                         filepath_12_2014,
                         filepath_01_2015,
                         filepath_02_2015,
                         filepath_03_2015,
                         filepath_04_2015,
                         filepath_05_2015,
                         filepath_06_2015,
                         filepath_07_2015,
                         filepath_08_2015,
                         filepath_09_2015,
                         filepath_10_2015,
                         filepath_11_2015,
                         filepath_12_2015,
                         filepath_01_2016,
                         filepath_02_2016,
                         filepath_03_2016,
                         filepath_04_2016,
                         filepath_05_2016,
                         filepath_06_2016,
                         filepath_07_2016,
                         filepath_08_2016,
                         filepath_09_2016,
                         filepath_10_2016,
                         filepath_11_2016,
                         filepath_12_2016
                    ]

### Getting the data for 105TK05 elemental for 105TK05 from 2014 to 216

In [54]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2014 to 2016 only. 

tk05_Pb = ['B', 'C', 'E', 'tk05_Pb']
tk05_Zn = ['B', 'C', 'F', 'tk05_Zn']
tk05_Cu = ['B', 'C', 'G', 'tk05_Cu']
tk05_Ni = ['B', 'C', 'H', 'tk05_Ni']
tk05_Co = ['B', 'C', 'I', 'tk05_Co']
tk05_Fe = ['B', 'C', 'J', 'tk05_Fe']
tk05_Mn = ['B', 'C', 'K', 'tk05_Mn']
tk05_Cr = ['B', 'C', 'L', 'tk05_Cr']
tk05_Ca = ['B', 'C', 'M', 'tk05_Ca']
tk05_Si = ['B', 'C', 'N', 'tk05_Si']
tk05_Al = ['B', 'C', 'O', 'tk05_Al']
tk05_Mg = ['B', 'C', 'P', 'tk05_Mg']

for i in range(0,len(dateList2014_2016)):
    
    print('{} start. time: {}'.format(dateList2014_2016[i], datetime.datetime.now()))
    monthLab = LabResults(dateList2014_2016[i], filepathList2014_2016[i])
    df_month = monthLab.generate_df_from_analysis('105TK05', tk05_Pb).join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Zn), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Cu), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Ni), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Co), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Fe), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Mn), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Cr), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Ca), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Si), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Al), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Mg), how='outer')
        
    df_month.to_excel(r'C:\Users\v.t.flores\Documents\ZnNi_Ratio_Proj_tk05_DataGather_Montly_{}.xlsx'.format(str(dateList2014_2016[i])))
    print('{} done. time: {}'.format(dateList2014_2016[i], datetime.datetime.now()))
    
# reset the variables to an empty list    
monthLab = []
df_month = []    
#-------------------------------------

2014-01-01 start. time: 2020-09-27 10:36:41.659791
2014-01-01 done. time: 2020-09-27 10:37:14.155197
2014-02-01 start. time: 2020-09-27 10:37:14.155197
2014-02-01 done. time: 2020-09-27 10:37:45.026029
2014-03-01 start. time: 2020-09-27 10:37:45.026029
2014-03-01 done. time: 2020-09-27 10:38:19.560092
2014-04-01 start. time: 2020-09-27 10:38:19.560092
2014-04-01 done. time: 2020-09-27 10:38:51.470872
2014-05-01 start. time: 2020-09-27 10:38:51.470872
2014-05-01 done. time: 2020-09-27 10:39:24.027111
2014-06-01 start. time: 2020-09-27 10:39:24.027111
2014-06-01 done. time: 2020-09-27 10:39:57.455993
2014-07-01 start. time: 2020-09-27 10:39:57.455993
2014-07-01 done. time: 2020-09-27 10:40:30.260173
2014-08-01 start. time: 2020-09-27 10:40:30.260173
2014-08-01 done. time: 2020-09-27 10:41:04.450001
2014-09-01 start. time: 2020-09-27 10:41:04.450999
2014-09-01 done. time: 2020-09-27 10:41:38.658697
2014-10-01 start. time: 2020-09-27 10:41:38.658697
2014-10-01 done. time: 2020-09-27 10:42:

In [55]:
# List corresponding to the cell location of each specific analysis type per sample source
# For 2017 to 2020 sept wk 1 only. 

tk05_Pb = ['B', 'C', 'E', 'tk05_Pb']
tk05_Zn = ['B', 'C', 'F', 'tk05_Zn']
tk05_Cu = ['B', 'C', 'G', 'tk05_Cu']
tk05_Ni = ['B', 'C', 'H', 'tk05_Ni']
tk05_Co = ['B', 'C', 'I', 'tk05_Co']
tk05_Fe = ['B', 'C', 'J', 'tk05_Fe']
tk05_Mn = ['B', 'C', 'K', 'tk05_Mn']
tk05_Cr = ['B', 'C', 'L', 'tk05_Cr']
tk05_Ca = ['B', 'C', 'M', 'tk05_Ca']
tk05_Si = ['B', 'C', 'N', 'tk05_Si']
tk05_Al = ['B', 'C', 'O', 'tk05_Al']
tk05_Mg = ['B', 'C', 'P', 'tk05_Mg']

for i in range(0,len(dateList20172020)):
    
    print('{} start. time: {}'.format(dateList20172020[i], datetime.datetime.now()))
    monthLab = LabResults(dateList20172020[i], filepathList20172020[i])
    df_month = monthLab.generate_df_from_analysis('105TK05', tk05_Pb).join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Zn), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Cu), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Ni), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Co), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Fe), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Mn), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Cr), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Ca), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Si), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Al), how='outer').join(
        monthLab.generate_df_from_analysis('105TK05', tk05_Mg), how='outer')
        
    df_month.to_excel(r'C:\Users\v.t.flores\Documents\ZnNi_Ratio_Proj_tk05_DataGather_Montly_{}.xlsx'.format(str(dateList20172020[i])))
    print('{} done. time: {}'.format(dateList20172020[i], datetime.datetime.now()))
    
# reset the variables to an empty list    
monthLab = []
df_month = []    
#-------------------------------------

2017-01-01 start. time: 2020-09-27 10:57:09.942976
2017-01-01 done. time: 2020-09-27 10:57:44.434736
2017-02-01 start. time: 2020-09-27 10:57:44.434736
2017-02-01 done. time: 2020-09-27 10:58:15.642727
2017-03-01 start. time: 2020-09-27 10:58:15.642727
2017-03-01 done. time: 2020-09-27 10:58:46.199369
2017-04-01 start. time: 2020-09-27 10:58:46.199369
2017-04-01 done. time: 2020-09-27 10:59:20.046525
2017-05-01 start. time: 2020-09-27 10:59:20.046525
2017-05-01 done. time: 2020-09-27 10:59:52.909969
2017-06-01 start. time: 2020-09-27 10:59:52.909969
2017-06-01 done. time: 2020-09-27 11:00:25.387651
2017-07-01 start. time: 2020-09-27 11:00:25.387651
2017-07-01 done. time: 2020-09-27 11:00:58.173862
2017-08-01 start. time: 2020-09-27 11:00:58.173862
2017-08-01 done. time: 2020-09-27 11:01:31.324052
2017-09-01 start. time: 2020-09-27 11:01:31.324052
2017-09-01 done. time: 2020-09-27 11:02:02.327363
2017-10-01 start. time: 2020-09-27 11:02:02.327363
2017-10-01 done. time: 2020-09-27 11:02: